In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Storing the datafile name in 'InputDataFile'.
InputDataFile = "Staffing+Data.xlsx"
InputDataFile

'Staffing+Data.xlsx'

In [4]:
# Reading the data from Excel workbook  and storing it in respective dataframes

Staff_Avail = pd.read_excel(InputDataFile, sheet_name="StaffAvailability")
Cost = pd.read_excel(InputDataFile, sheet_name="Cost")
Trans_Cost = pd.read_excel(InputDataFile, sheet_name="ServiceRate")
Mg_Demand_Dist = pd.read_excel(InputDataFile, sheet_name="DemandData")


In [5]:
# Checking 'Staff_Avail' dataframe
Staff_Avail.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


## Data pre-processing 

In [6]:
# Create the required Python data structures for indexes and parameters

# Converting each parameter to dictionary

# Demand Data
Demand = Mg_Demand_Dist.set_index(['State','Month'])['Demand'].to_dict()

# FTE Salary from the data
FTE_Monthly_Salary = Cost.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Unit Outsourcing Cost
UnitOutSourcingCost = Cost.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

# Number of Applications handled by an employee when he/she is working 100%
FTE_AppServiceRate = Trans_Cost.iloc[0,0]

# Staff Availability for serving the Insurance Applications
Staff_Avail_LB = Staff_Avail.set_index(['State','Month'])['LB'].to_dict() # Worst Case
Staff_Avail_UB = Staff_Avail.set_index(['State','Month'])['UB'].to_dict() # Best Case
Staff_Avail_Per = Staff_Avail.set_index(['State','Month'])['StaffAvPer'].to_dict() # Actual Case


# Indexes
# Storing unique locations (i.e. States) in 'InsCom_Location'
InsCom_Location = Cost['State'].unique()

# Storing unique months in 'Month'
Month = Cost['Month'].unique()

In [7]:
# Checking Worst Case Staff Availability
Staff_Avail_LB

{('A', 'Jan'): 0.7,
 ('A', 'Feb'): 0.65,
 ('A', 'Mar'): 0.7,
 ('A', 'Apr'): 0.75,
 ('A', 'May'): 0.7,
 ('A', 'Jun'): 0.65,
 ('A', 'Jul'): 0.6,
 ('A', 'Aug'): 0.65,
 ('A', 'Sep'): 0.7,
 ('A', 'Oct'): 0.65,
 ('A', 'Nov'): 0.6,
 ('A', 'Dec'): 0.6,
 ('B', 'Jan'): 0.7,
 ('B', 'Feb'): 0.65,
 ('B', 'Mar'): 0.7,
 ('B', 'Apr'): 0.75,
 ('B', 'May'): 0.7,
 ('B', 'Jun'): 0.65,
 ('B', 'Jul'): 0.6,
 ('B', 'Aug'): 0.65,
 ('B', 'Sep'): 0.7,
 ('B', 'Oct'): 0.65,
 ('B', 'Nov'): 0.6,
 ('B', 'Dec'): 0.6,
 ('C', 'Jan'): 0.7,
 ('C', 'Feb'): 0.65,
 ('C', 'Mar'): 0.7,
 ('C', 'Apr'): 0.75,
 ('C', 'May'): 0.7,
 ('C', 'Jun'): 0.65,
 ('C', 'Jul'): 0.6,
 ('C', 'Aug'): 0.65,
 ('C', 'Sep'): 0.7,
 ('C', 'Oct'): 0.65,
 ('C', 'Nov'): 0.6,
 ('C', 'Dec'): 0.6}

In [8]:
# Checking Best Case Staff Availability
Staff_Avail_UB

{('A', 'Jan'): 0.9,
 ('A', 'Feb'): 0.85,
 ('A', 'Mar'): 0.8,
 ('A', 'Apr'): 0.85,
 ('A', 'May'): 0.85,
 ('A', 'Jun'): 0.8,
 ('A', 'Jul'): 0.75,
 ('A', 'Aug'): 0.85,
 ('A', 'Sep'): 0.9,
 ('A', 'Oct'): 0.8,
 ('A', 'Nov'): 0.75,
 ('A', 'Dec'): 0.7,
 ('B', 'Jan'): 0.9,
 ('B', 'Feb'): 0.85,
 ('B', 'Mar'): 0.8,
 ('B', 'Apr'): 0.85,
 ('B', 'May'): 0.85,
 ('B', 'Jun'): 0.8,
 ('B', 'Jul'): 0.75,
 ('B', 'Aug'): 0.85,
 ('B', 'Sep'): 0.9,
 ('B', 'Oct'): 0.8,
 ('B', 'Nov'): 0.75,
 ('B', 'Dec'): 0.7,
 ('C', 'Jan'): 0.9,
 ('C', 'Feb'): 0.85,
 ('C', 'Mar'): 0.8,
 ('C', 'Apr'): 0.85,
 ('C', 'May'): 0.85,
 ('C', 'Jun'): 0.8,
 ('C', 'Jul'): 0.75,
 ('C', 'Aug'): 0.85,
 ('C', 'Sep'): 0.9,
 ('C', 'Oct'): 0.8,
 ('C', 'Nov'): 0.75,
 ('C', 'Dec'): 0.7}

In [9]:
# Checking Actual Case Staff Availability
Staff_Avail_Per

{('A', 'Jan'): 0.81,
 ('A', 'Feb'): 0.76,
 ('A', 'Mar'): 0.75,
 ('A', 'Apr'): 0.8,
 ('A', 'May'): 0.78,
 ('A', 'Jun'): 0.73,
 ('A', 'Jul'): 0.68,
 ('A', 'Aug'): 0.76,
 ('A', 'Sep'): 0.81,
 ('A', 'Oct'): 0.73,
 ('A', 'Nov'): 0.68,
 ('A', 'Dec'): 0.65,
 ('B', 'Jan'): 0.81,
 ('B', 'Feb'): 0.76,
 ('B', 'Mar'): 0.75,
 ('B', 'Apr'): 0.8,
 ('B', 'May'): 0.78,
 ('B', 'Jun'): 0.73,
 ('B', 'Jul'): 0.68,
 ('B', 'Aug'): 0.76,
 ('B', 'Sep'): 0.81,
 ('B', 'Oct'): 0.73,
 ('B', 'Nov'): 0.68,
 ('B', 'Dec'): 0.65,
 ('C', 'Jan'): 0.81,
 ('C', 'Feb'): 0.76,
 ('C', 'Mar'): 0.75,
 ('C', 'Apr'): 0.8,
 ('C', 'May'): 0.78,
 ('C', 'Jun'): 0.73,
 ('C', 'Jul'): 0.68,
 ('C', 'Aug'): 0.76,
 ('C', 'Sep'): 0.81,
 ('C', 'Oct'): 0.73,
 ('C', 'Nov'): 0.68,
 ('C', 'Dec'): 0.65}

In [10]:
# Displaying the locations
InsCom_Location

array(['A', 'B', 'C'], dtype=object)

In [11]:
# Converting InsCom_Location to list
InsCom_Location.tolist()

['A', 'B', 'C']

In [12]:
# Displaying the months
Month

array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec'], dtype=object)

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



**Mathematical formulations are included in the report.**

**Note:** You can write the mathematical formulation directly in your report.

# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [13]:
# Creating a model instance
model = ConcreteModel()

In [14]:
# Define Pyomo sets and Parameters

In [15]:
# Define Pyomo sets
model.i = Set(initialize=InsCom_Location.tolist(), doc="States")
model.j = Set(initialize=Month.tolist(), doc="Months")

In [16]:
# Define parameters for demand, FTE salaries, outsourcing cost and Average Staff Availability

model.d = Param(model.i, model.j, initialize=Demand, doc="Demand")
model.fs = Param(model.i, model.j, initialize=FTE_Monthly_Salary, doc="FTE_Monthly_Salary")
model.osc = Param(model.i, model.j, initialize=UnitOutSourcingCost, doc="OutSourcingCost")
model.sap = Param(model.i, model.j, initialize=Staff_Avail_Per, doc="StaffAvailPercent")

In [17]:
# FTE Application Serving Rate per month when working 100%

model.fsr = Param(initialize=FTE_AppServiceRate, doc="FTE App Service Rate")

In [18]:
# Parameter for the number of applications that can be processed for the given staff availability

def c_FTEAPP(model, i, j):
    return model.fsr * model.sap[i,j]

model.FTEAPPNUM = Param(model.i, model.j, initialize=c_FTEAPP, doc="NUMBER OF THE FTE APP")

In [19]:
# Parameters for restriction

# Regulatory constraint for A i.e. 30%

def Reg_A(model, i, j):
    return 0.3 * model.d[i,j]

model.RegA = Param(model.i, model.j, initialize=Reg_A, doc='RegRest_A')

In [20]:
# Regulatory constraint for B i.e. 40%

def Reg_B(model, i, j):
    return 0.4 * model.d[i,j]

model.RegB = Param(model.i, model.j, initialize=Reg_B, doc='RegRest_B')

In [21]:
# Decision variables
model.x = Var(model.i, model.j, domain=NonNegativeReals, doc="Number of FTE")
model.y = Var(model.i, model.j, domain=NonNegativeIntegers, doc="Number of Outsourced App")

In [22]:
# Demand Constraint
model.demand_constraint = ConstraintList()
for i in model.i:
    for j in model.j:
        model.demand_constraint.add(expr= model.x[i,j] * model.FTEAPPNUM[i,j] + model.y[i,j] == model.d[i,j])


In [23]:
# Regulatory Constraints
model.regulatoryCons = ConstraintList()
for i in model.i:
    for j in model.j:
        if i == 'A':
            model.regulatoryCons.add(expr= model.y['A',j]<=model.RegA['A',j])
        elif i=='B':
            model.regulatoryCons.add(expr= model.y['B',j]<=model.RegB['B',j])

In [24]:
# Objective function
def objective_rule(model):
    return sum(model.x[i,j]*model.fs[i,j] for i in model.i for j in model.j) +\
           sum(model.osc[i,j]*model.y[i,j] for i in model.i for j in model.j)

# Objective is minimize the cost
model.objective = Objective(rule=objective_rule, sense=minimize, doc="Define Objective Function")

In [25]:
# Invoking the solver
Result_Cost = SolverFactory("glpk").solve(model)
Result_Cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.1212773323059082
# ----------------------------------------------------------
#   Solution Information
# -------------

In [26]:
print(model.x['A','Dec'].value)

139.576923076923


In [27]:
print(model.y['A','Dec'].value) 

1554.0


In [28]:
# Print the value of the objective function

Total_Cost = model.objective.expr()
Total_Cost

17962336.448769882

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.  - **YES**

In [29]:
# Creating dataframe for the results

FTE_Staff = []

for i in InsCom_Location:
    for j in Month:
        d = model.d[i,j]
        s = model.x[i,j].value
        o = model.y[i,j].value
        ec = model.x[i,j].value*model.fs[i,j]+model.osc[i,j]*model.y[i,j].value
        cpa = round(ec/d,2)
        FTE_Staff.append([i,j,d,s,o,ec,cpa])
        
FTE_Staff = pd.DataFrame(FTE_Staff, columns=['State',"Month","Demand","Staff_FTE","App_Outsourced","Estimated Cost Per Month","Cost Per Application"])

In [30]:
#Displaying actual scenario results
FTE_Staff.head(10)

,State,Month,Demand,Staff_FTE,App_Outsourced,Estimated Cost Per Month,Cost Per Application
0,A,Jan,5240,161.728395,0.0,808641.975309,154.32
1,A,Feb,4878,160.460526,0.0,802302.631579,164.47
2,A,Mar,5942,198.066667,0.0,990333.333333,166.67
3,A,Apr,2297,71.781250,0.0,358906.250000,156.25
4,A,May,1992,63.846154,0.0,319230.769231,160.26
5,A,Jun,2275,77.910959,0.0,389554.794521,171.23
6,A,Jul,5334,137.279412,1600.0,974397.058824,182.68
7,A,Aug,3371,110.888158,0.0,554440.789474,164.47
8,A,Sep,3759,116.018519,0.0,580092.592593,154.32
9,A,Oct,3529,120.856164,0.0,604280.821918,171.23


In [31]:
# For Actual Scenario,
Average_Cost_Per_App = round(FTE_Staff['Cost Per Application'].mean(),2)
print("Average Cost (in $) Per Application (Actual Case): ", Average_Cost_Per_App)

Average Cost (in $) Per Application (Actual Case):  155.88


In [32]:
# Calculating and printing optimal number of staff members for the actual scenario
Total_Staff_actual = sum(FTE_Staff['Staff_FTE'])
print("Optimal Number of Staff Members (Actual Case): ", Total_Staff_actual)

Optimal Number of Staff Members (Actual Case):  3085.50849262589


In [33]:
#Calculating and printing the percentage of outsourced applications for the actual scenario
Total_App_Outsourced_actual = sum(FTE_Staff['App_Outsourced'])
Total_Demand_actual = sum(FTE_Staff['Demand'])
Percentage_App_Outsourced_actual = round((Total_App_Outsourced_actual/Total_Demand_actual)*100, 2)

print("Total Number of Outsourced Applications (Actual Case): ", Total_App_Outsourced_actual)
print("Total Demand of Applications: ", Total_Demand_actual)
print("Percentage of Outsourced Applications (Actual Case): ", Percentage_App_Outsourced_actual)


Total Number of Outsourced Applications (Actual Case):  20749.0
Total Demand of Applications:  113294
Percentage of Outsourced Applications (Actual Case):  18.31


In [34]:
# Writing the results in to an Excel sheet

from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# Create excel 'write' object
write = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')

# Assigning the workbook to the 'write' object
write.book = book

# Writing the 'FTE_Staff' dataframe to 'Output_Actual' excel sheet.
FTE_Staff.to_excel(write, sheet_name='Output_Actual')

# Save the excel file
write.save()
print('Staffing+Data.xlsx excel file has been updated successfully.')


Staffing+Data.xlsx excel file has been updated successfully.


# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [35]:
# Creating a model instance
model_worst = ConcreteModel()

In [36]:
# Define Pyomo sets and Parameters
model_worst.i = Set(initialize=InsCom_Location.tolist(), doc="States")
model_worst.j = Set(initialize=Month.tolist(), doc="Months")

In [37]:
# Define parameters for demand, FTE salaries, outsourcing cost and Average Staff Availability
model_worst.d = Param(model_worst.i, model_worst.j, initialize=Demand, doc="Demand")
model_worst.fs = Param(model_worst.i, model_worst.j, initialize=FTE_Monthly_Salary, doc="FTE_Monthly_Salary")
model_worst.osc = Param(model_worst.i, model_worst.j, initialize=UnitOutSourcingCost, doc="OutSourcingCost")
model_worst.sap = Param(model_worst.i, model_worst.j, initialize=Staff_Avail_LB, doc="StaffAvailPercent")

In [38]:
# Number of Applications handled by an employee when he/she is working 100%
model_worst.fsr = Param(initialize=FTE_AppServiceRate, doc="FTE App Service Rate")

In [39]:
# Parameter for the no of Applications that can be processed for the given staff availability

def c_FTEAPP(model1, i, j):
    return model1.fsr * model1.sap[i,j]

model_worst.FTEAPPNUM = Param(model_worst.i, model_worst.j, initialize=c_FTEAPP, doc="NUMBER OF THE FTE APP")

In [40]:
# Parameters for restriction

# Regulatory constraint for A i.e. 30%

def Reg_A(model1, i, j):
    return 0.3 * model1.d[i,j]

model_worst.RegA = Param(model_worst.i, model_worst.j, initialize=Reg_A, doc='RegRest_A')

In [41]:
# Regulatory constraint for B i.e. 40%

def Reg_B(model1, i, j):
    return 0.4 * model1.d[i,j]

model_worst.RegB = Param(model_worst.i, model_worst.j, initialize=Reg_B, doc='RegRest_B')

In [42]:
# Decision variables
model_worst.x = Var(model_worst.i, model_worst.j, domain=NonNegativeReals, doc="Number of FTE")
model_worst.y = Var(model_worst.i, model_worst.j, domain=NonNegativeIntegers, doc="Number of Outsourced App")

In [43]:
# Constraints
# Demand Constraint
model_worst.demand_constraint = ConstraintList()
for i in model_worst.i:
    for j in model_worst.j:
        model_worst.demand_constraint.add(expr= model_worst.x[i,j] * model_worst.FTEAPPNUM[i,j] + model_worst.y[i,j] == model_worst.d[i,j])

In [44]:
# Regulatory Constraints
model_worst.regulatoryCons = ConstraintList()
for i in model_worst.i:
    for j in model_worst.j:
        if i == 'A':
            model_worst.regulatoryCons.add(expr= model_worst.y['A',j]<=model_worst.RegA['A',j])
        elif i=='B':
            model_worst.regulatoryCons.add(expr= model_worst.y['B',j]<=model_worst.RegB['B',j])

In [45]:
# Objective function
def objective_rule(model1):
    return sum(model1.x[i,j]*model1.fs[i,j] for i in model1.i for j in model1.j) +\
           sum(model1.osc[i,j]*model1.y[i,j] for i in model1.i for j in model1.j)

model_worst.objective = Objective(rule=objective_rule, sense=minimize, doc="Define Objective Function")


In [46]:
# Invoking the solver
Result_Worst_cost = SolverFactory("glpk").solve(model_worst)
Result_Worst_cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 19599482.5167888
  Upper bound: 19599482.5167888
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.10502386093139648
# ----------------------------------------------------------
#   Solution Information
# ------------

In [47]:
# Print the value of the objective function
Total_Worst_Cost = model_worst.objective.expr()
Total_Worst_Cost

19599482.516788766

**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability. - **YES**

In [48]:
# Creating dataframe for the results
# Creating dataframe 'FTE_Staff_Worst' and storing worst case results

FTE_Staff_Worst = []

for i in InsCom_Location:
    for j in Month:
        d = model_worst.d[i,j]
        s = model_worst.x[i,j].value
        o = model_worst.y[i,j].value
        ec = model_worst.x[i,j].value*model_worst.fs[i,j]+model_worst.osc[i,j]*model_worst.y[i,j].value
        cpa = round(ec/d,2)
        FTE_Staff_Worst.append([i,j,d,s,o,ec,cpa])
                
FTE_Staff_Worst = pd.DataFrame(FTE_Staff_Worst, columns=['State',"Month","Demand","Staff_FTE","App_Outsourced","Estimated Cost Per Month","Cost Per Application"])


In [49]:
#Displaying worst-case scenario results

FTE_Staff_Worst.head(10)

,State,Month,Demand,Staff_FTE,App_Outsourced,Estimated Cost Per Month,Cost Per Application
0,A,Jan,5240,187.142857,0.0,9.357143e+05,178.57
1,A,Feb,4878,131.346154,1463.0,9.200708e+05,188.62
2,A,Mar,5942,212.214286,0.0,1.061071e+06,178.57
3,A,Apr,2297,76.566667,0.0,3.828333e+05,166.67
4,A,May,1992,71.142857,0.0,3.557143e+05,178.57
5,A,Jun,2275,61.269231,682.0,4.291062e+05,188.62
6,A,Jul,5334,155.583333,1600.0,1.065917e+06,199.83
7,A,Aug,3371,90.769231,1011.0,6.358262e+05,188.62
8,A,Sep,3759,134.250000,0.0,6.712500e+05,178.57
9,A,Oct,3529,95.038462,1058.0,6.656323e+05,188.62


In [50]:
# Writing the worst case results into an Excel sheet

from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# Create excel 'write' object
write = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')

# Assigning the workbook to the 'write' object
write.book = book

# Writing the 'FTE_Staff_Worst' dataframe to 'Output_Worst' excel sheet.
FTE_Staff_Worst.to_excel(write, sheet_name='Output_Worst')

# Save the excel file
write.save()
print('Staffing+Data.xlsx excel file has been updated successfully with the Worst Case Results.')

Staffing+Data.xlsx excel file has been updated successfully with the Worst Case Results.


In [51]:
#Calculating and printing the optimal number of staff members for worst-case scenario
Total_Staff_Worst = sum(FTE_Staff_Worst['Staff_FTE'])
print("Optimal Number of Staff Members (Worst Case): ", Total_Staff_Worst)

Optimal Number of Staff Members (Worst Case):  2761.2709706959704


#### 3.1.2 Percentage of outsourced applications 

In [52]:
# For Worst Case Scenario,
#Calculating and printing the percentage of outsourced applications for the worst-case scenario

Total_App_Outsourced_Worst = sum(FTE_Staff_Worst['App_Outsourced'])
Total_Demand_Worst = sum(FTE_Staff_Worst['Demand'])
Percentage_App_Outsourced_Worst = round((Total_App_Outsourced_Worst/Total_Demand_Worst)*100, 2)

print("Total Number of Outsourced Applications (Worst Case): ", Total_App_Outsourced_Worst)
print("Total Demand of Applications: ", Total_Demand_Worst)
print("Percentage of Outsourced Applications (Worst Case): ", Percentage_App_Outsourced_Worst)

Total Number of Outsourced Applications (Worst Case):  39808.0
Total Demand of Applications:  113294
Percentage of Outsourced Applications (Worst Case):  35.14


#### 3.1.3 Average cost per application

In [53]:
# For Worst Case Scenario,
#Calculating and printing average cost per application for worst-case scenario
Average_Cost_Per_App_Worst = round(FTE_Staff_Worst['Cost Per Application'].mean(),2)
print("Average Cost (in $) Per Application (Worst Case): ", Average_Cost_Per_App_Worst)

Average Cost (in $) Per Application (Worst Case):  169.57


### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [54]:
# Creating a model instance
model_best = ConcreteModel()

In [55]:
# Define Pyomo sets and Parameters
model_best.i = Set(initialize=InsCom_Location.tolist(), doc="States")
model_best.j = Set(initialize=Month.tolist(), doc="Months")

In [56]:
# Define parameters for demand, FTE salaries, outsourcing cost and Average Staff Availability
model_best.d = Param(model_best.i, model_best.j, initialize=Demand, doc="Demand")
model_best.fs = Param(model_best.i, model_best.j, initialize=FTE_Monthly_Salary, doc="FTE_Monthly_Salary")
model_best.osc = Param(model_best.i, model_best.j, initialize=UnitOutSourcingCost, doc="OutSourcingCost")
model_best.sap = Param(model_best.i, model_best.j, initialize=Staff_Avail_UB, doc="StaffAvailPercent")

In [57]:
# Number of Applications handled by an employee when he/she is working 100%
model_best.fsr = Param(initialize=FTE_AppServiceRate, doc="FTE App Service Rate")

In [58]:
# Parameter for the number of applications that can be processed for the given best case staff availability

def c_FTEAPP(model2, i, j):
    return model2.fsr * model2.sap[i,j]

model_best.FTEAPPNUM = Param(model_best.i, model_best.j, initialize=c_FTEAPP, doc="NUMBER OF THE FTE APP")

In [59]:
# Parameters for restriction

# Regulatory constraint for A i.e. 30%
def Reg_A(model2, i, j):
    return 0.3 * model2.d[i,j]

model_best.RegA = Param(model_best.i, model_best.j, initialize=Reg_A, doc='RegRest_A')

In [60]:
# Regulatory constraint for B i.e. 40%
def Reg_B(model2, i, j):
    return 0.4 * model2.d[i,j]

model_best.RegB = Param(model_best.i, model_best.j, initialize=Reg_B, doc='RegRest_B')

In [61]:
# Decision variables
model_best.x = Var(model_best.i, model_best.j, domain=NonNegativeReals, doc="Number of FTE")
model_best.y = Var(model_best.i, model_best.j, domain=NonNegativeIntegers, doc="Number of Outsourced App")

In [62]:
# Constraints
# Demand Constraint
model_best.demand_constraint = ConstraintList()
for i in model_best.i:
    for j in model_best.j:
        model_best.demand_constraint.add(expr= model_best.x[i,j] * model_best.FTEAPPNUM[i,j] + model_best.y[i,j] == model_best.d[i,j])


In [63]:
# Regulatory Constraints
model_best.regulatoryCons = ConstraintList()
for i in model_best.i:
    for j in model_best.j:
        if i == 'A':
            model_best.regulatoryCons.add(expr= model_best.y['A',j]<=model_best.RegA['A',j])
        elif i=='B':
            model_best.regulatoryCons.add(expr= model_best.y['B',j]<=model_best.RegB['B',j])

In [64]:
# Objective Function
def objective_rule(model2):
    return sum(model2.x[i,j]*model2.fs[i,j] for i in model2.i for j in model2.j) +\
           sum(model2.osc[i,j]*model2.y[i,j] for i in model2.i for j in model2.j)

model_best.objective = Objective(rule=objective_rule, sense=minimize, doc="Define Objective Function")


In [65]:
# Invoking the solver
Result_Best_Cost = SolverFactory("glpk").solve(model_best)
Result_Best_Cost.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16527535.6379357
  Upper bound: 16527535.6379357
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.09502005577087402
# ----------------------------------------------------------
#   Solution Information
# ------------

In [66]:
# Print the value of the objective function
Total_Best_Cost = model_best.objective.expr()
Total_Best_Cost

16527535.63793573

**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability. - **YES**

In [67]:
# Creating dataframe for the results
# Creating dataframe 'FTE_Staff_Best' and storing best case results

FTE_Staff_Best = []

for i in InsCom_Location:
    for j in Month:
        d = model_best.d[i,j]
        s = model_best.x[i,j].value
        o = model_best.y[i,j].value
        ec = model_best.x[i,j].value*model_best.fs[i,j]+model_best.osc[i,j]*model_best.y[i,j].value
        cpa = round(ec/d,2)
        FTE_Staff_Best.append([i,j,d,s,o,ec,cpa])
        
FTE_Staff_Best = pd.DataFrame(FTE_Staff_Best, columns=['State',"Month","Demand","Staff_FTE","App_Outsourced","Estimated Cost Per Month","Cost Per Application"])

In [68]:
#Displaying best-case scenario results
FTE_Staff_Best.head(10)

,State,Month,Demand,Staff_FTE,App_Outsourced,Estimated Cost Per Month,Cost Per Application
0,A,Jan,5240,145.555556,0.0,727777.777778,138.89
1,A,Feb,4878,143.470588,0.0,717352.941176,147.06
2,A,Mar,5942,185.687500,0.0,928437.500000,156.25
3,A,Apr,2297,67.558824,0.0,337794.117647,147.06
4,A,May,1992,58.588235,0.0,292941.176471,147.06
5,A,Jun,2275,71.093750,0.0,355468.750000,156.25
6,A,Jul,5334,177.800000,0.0,889000.000000,166.67
7,A,Aug,3371,99.147059,0.0,495735.294118,147.06
8,A,Sep,3759,104.416667,0.0,522083.333333,138.89
9,A,Oct,3529,110.281250,0.0,551406.250000,156.25


In [69]:
# Writing the best case results into an Excel sheet

from openpyxl import load_workbook
book = load_workbook("Staffing+Data.xlsx")

# Create excel 'write' object
write = pd.ExcelWriter("Staffing+Data.xlsx", engine = 'openpyxl')

# Assigning the workbook to the 'write' object
write.book = book

# Writing the 'FTE_Staff' dataframe to 'Output_Actual' excel sheet.
FTE_Staff_Best.to_excel(write, sheet_name='Output_Best')

# Save the excel file
write.save()
print('Staffing+Data.xlsx excel file has been updated successfully with the Best Case Results.')

Staffing+Data.xlsx excel file has been updated successfully with the Best Case Results.


In [70]:
#Calculating and printing the optimal number of staff members for best-case scenario

Total_Staff_Best = sum(FTE_Staff_Best['Staff_FTE'])
print("Optimal Number of Staff Members (Best Case): ", Total_Staff_Best)

Optimal Number of Staff Members (Best Case):  3343.189524976659


#### 3.2.2 Percentage of outsourced applications

In [71]:
# For Best Case Scenario,
#Calculating and printing the percentage of outsourced applications for the best-case scenario
Total_App_Outsourced_Best = sum(FTE_Staff_Best['App_Outsourced'])
Total_Demand_Best = sum(FTE_Staff_Best['Demand'])
Percentage_App_Outsourced_Best = round((Total_App_Outsourced_Best/Total_Demand_Best)*100, 2)

print("Total Number of Outsourced Applications (Best Case): ", Total_App_Outsourced_Best)
print("Total Demand of Applications: ", Total_Demand_Best)
print("Percentage of Outsourced Applications (Best Case): ", Percentage_App_Outsourced_Best)

Total Number of Outsourced Applications (Best Case):  4652.0
Total Demand of Applications:  113294
Percentage of Outsourced Applications (Best Case):  4.11


#### 3.2.3 Average cost per application

In [72]:
# For Best Case Scenario,
#Calculating and printing the average cost per application for best-case scenario
Average_Cost_Per_App_Best = round(FTE_Staff_Best['Cost Per Application'].mean(),2)
print("Average Cost (in $) Per Application (Best Case): ", Average_Cost_Per_App_Best)

Average Cost (in $) Per Application (Best Case):  143.42


# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.

**Charts are added directly in the report.**